In [1]:
import numpy as np 
import pandas as pd

#скачиываем файлы из зип файла и далее сохраняем в корневую директорию data_root

import os 
print(os.listdir('../input'),'\n') 

import zipfile
with zipfile.ZipFile('../input/platesv2/plates.zip', 'r') as zip_obj:
        zip_obj.extractall('/kaggle/working/') 
print(os.listdir('/kaggle/working/'))

data_root = '/kaggle/working/plates/' 
print(data_root)
print(os.listdir(data_root))


FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: '../input'

In [ ]:
import shutil 
from tqdm import tqdm

#у нас есть папки тест и трейн, тогда берём из трейна каждую шестую фотку в валидацию, чтобы контролировать качество

train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name) 
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

In [ ]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy

from torchvision import transforms, models
#проделаем аугментацию:
train_transforms = transforms.Compose([      
    transforms.RandomResizedCrop(224),     #вырезает из изображение прямоуг и ресайзит его до размера 224
    transforms.RandomHorizontalFlip(),     #отзеркалили
    transforms.ToTensor(),                 #в тензор
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  #отнормировали для резнета
])

val_transforms = transforms.Compose([    #все трансформации будут подряд
    transforms.Resize((224, 224)),       #ужмём до размера 224 на 224, т.к. на таком размере обучен ResNet
    transforms.ToTensor(),               #изображение переводим в тензор
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  #ResNet работал на предобработанных изображений, чтобы сделать такие же, отнормируем их по константам(с сайта pyTorch)
])

#чтобы итерироваться по папкам и доставать оттуда картинки с указателем мы используем ImageFolder из TorchVision
#эта функция подаёт изображение и его метку, то есть в папке есть папка clean и dirty, тогда фотку она передаёт с меткой папки
#второй аргумент в ImageFolder нужна, чтобы получать тензор, поэтому притменяется трансформация

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

#объект DataLoader принимает на вход ImageFolder, размер батча, можно перемашать датасет, аргумент num_workers позволяет 
#распаралелить процесс по указанному количеству тредов

batch_size = 8
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

In [ ]:
#если хотим посмотреть всё-так, что же у нас в даталоадаре, вот код 

#то, что у нас было
X_batch, y_batch = next(iter(train_dataloader))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
plt.imshow(X_batch[0].permute(1, 2, 0).numpy() * std + mean);

#что стало 
def show_input(input_tensor, title=''):
    image = input_tensor.permute(1, 2, 0).numpy()
    image = std * image + mean
    plt.imshow(image.clip(0, 1))
    plt.title(title)
    plt.show()
    plt.pause(0.001)

X_batch, y_batch = next(iter(train_dataloader))

for x_item, y_item in zip(X_batch, y_batch):
    show_input(x_item, title=class_names[y_item])

In [ ]:
#функция трейн модел идёт по батчам, если фаза обучения, то она делает шаг град спуска, а если валидация,
#то вычисляет результат. В train_model все те переменные, описанные ниже.

def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader #даёт батчи из трейн датасета
                scheduler.step()              #делаем шаг
                model.train()                 #переводим модель в фазу тренировки
            else:
                dataloader = val_dataloader
                model.eval()                  #Тут могут изменяться какие-то параметры, и чтобы нейронка не менялась 
                                              #ставим eval, чтобы с течением времени ничего не менялось
            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader): #из даталоадера берём батч и метку
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                #Если фаща train, то активируем все незаморож град, а на валидации ничего не считается и не засор память
                with torch.set_grad_enabled(phase == 'train'):  
                    preds = model(inputs)          #получаем выходу моделей на наших инпутах 
                    loss_value = loss(preds, labels) #посчитали лосс-функцию по предсказанием
                    preds_class = preds.argmax(dim=1) #берём нейрон с макс активации, чтобы понять что за класс предсказан

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                #подсчёт статистики лоссов и точности с течением эпох (делим на длину даталодера(кол-во батчей в эпохе))
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model

In [ ]:
#делаем модель, использовать мы будем уже готовую модель ResNet и дообучаться на ней, ResNet18 значит, что у неё 18 слоёв
#обычно берут и больше. Параметр pretrained=True отвечает за то, что мы берём уже готовые обученные веса
model = models.resnet18(pretrained=True)

#Так как выборка у нас маленькая (всего 40 фоток в трейне), тогда хотим изменять последний слой, т.к глубоко идти нет смысла
#model.parameters - итератор по слоям, и говорим, что по параметрам этих слоёв вычисление градиента не требуется
for param in model.parameters():
    param.requires_grad = False

#меняем выход, так как классификация состоит только из двух вариантов 
model.fc = torch.nn.Linear(model.fc.in_features, 2)  #последний слой получается единственный размороженный

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

loss = torch.nn.CrossEntropyLoss() #на вход получает не вероятности после SoftMax, а активации нейронов (то есть до)
optimizer = torch.optim.Adam(model.parameters(), amsgrad = True, lr=1.0e-3)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1) #изменяет шаг градиентного спуска

In [ ]:
#Костыль ImageFolder, который не может обработать путь к папке в которой уже сразу лежат изображения
#Копируем всю папку test в директорию test\unknown
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

In [ ]:
#Мы не знаем, какие ID, какие названия изображения у нас генерируется, когда мы просим у DataLoader -- "дай нам следующий батч".
#Они по алфавиту идут, по дате создания, или просто случайным образом -- непонятно.
#Поэтому нам нужно переписать немножко ImageFolder, чтобы он нам отдавал не просто tuple, с самим изображением и его меткой, а ещё, чтобы он отдавал имя, ну, либо -- путь к изображению.
class ImageFolderWithPaths(torchvision.datasets.ImageFolder): #Создаем класс, он наследуется от ImageFolder, но изменяет его функцию get_item
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index) #Дополняем original_tuple путем для файла (.__getitem__(index))
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [ ]:
model.eval() #Переводим модель в состояние eval

test_predictions = []  #Создаем пустой список предсказания 
test_img_paths = [] #Пути до изображения
for inputs, labels, paths in tqdm(test_dataloader): #Цикл по test_dataloader inputs - батч с изображением, lable - тут none, paths - пути до изображения  
    inputs = inputs.to(device) 
    labels = labels.to(device)  
    with torch.set_grad_enabled(False):
        preds = model(inputs) # Считаем предикшены
    test_predictions.append(
        torch.nn.functional.softmax(preds, dim=1)[:,1].data.cpu().numpy()) #С помощью torch.nn.functional.softmax получаем вероятности, для первого класса [:,1], пеереводим тензор в .data, на .cpu(), в numpy 
    test_img_paths.extend(paths)
test_predictions = np.concatenate(test_predictions)

In [ ]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})

In [ ]:
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.head(n=6)